In [ ]:
import numpy as n
from scipy.optimize import root
import Properties as prop
import pandas as pd

import json, CoolProp.CoolProp as CP
CP.set_config_string(CP.ALTERNATIVE_REFPROP_PATH, 'C:\\Program Files (x86)\\REFPROP\\')



calctolerance=10**-5
calcmethod = "hybr"
KPD=0.99

#Таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#Таблица частичной нагрузки (входные данные)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
###########################################################
class heatex:
    def __init__(self,stream11,stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P021 = water_streams0.at[stream21,'P']
        self.P022 = water_streams0.at[stream22,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.G2   = water_streams.at[stream21,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P21  = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P021)
        T022   = prop.water_T_HP(self.H022,self.P022)
        dTmin0 = min(T011-T022,T012-T021)
        dTmax0 = max(T011-T022,T012-T021)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        T02av  = (T021+T022)/2
        P02av  = (self.P021+self.P022)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        ro02av = prop.water_D_PT(P02av,T02av)
        ro021  = prop.water_DSS_P(self.P021)
        ro21   = prop.water_DSS_P(self.P21)
        ddp    = (ro21/ro021)*((self.G02/self.G2)**2)
        P22    = self.P21 - ((self.P021-self.P022)/ddp)
        P12 = self.P1
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P21)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = self.H21 + (Q/self.G2)
            T22 = prop.water_T_HP(H22,P22)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = self.H21 + (Q/self.G2)
        T22 = prop.water_T_HP(H22,P22)
        return [T12,P12,H12,self.G1,T22,P22,H22,self.G2,Q]
class vaporND:
    def __init__(self, stream11, stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P02  = water_streams0.at[stream21,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P2   = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P02)
        T022   = prop.water_T_HP(self.H022,self.P02)
        T0np   = prop.water_TSS_P(self.P2)
        dTmin0 = min(T011-T0np,T012-T0np)
        dTmax0 = max(T011-T0np,T012-T0np)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P2)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = prop.water_HSS_P(self.P2)
            T22 = prop.water_T_HP(H22,self.P2)
            Dvd = water_streams.at['PEVD-DROS','G']
            Hvd =prop.water_HSW_P(self.P2)
            G2 = (Q-Dvd*(Hvd-self.H21))/(H22-self.H21)          ####!
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = prop.water_HSS_P(self.P2)
        T22 = prop.water_T_HP(H22,self.P2)
        G2 = Q/(H22-self.H21)
        return [T12,self.P1,H12,self.G1,T22,self.P2,H22,G2,Q]
class vapor:
    def __init__(self, stream11, stream12,stream21,stream22):
        self.Gas_composition0={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.Gas_composition={'N2':100*gas_streams.at[stream11,'N2'],'O2':100*gas_streams.at[stream11,'O2'],'CO2':100*gas_streams.at[stream11,'CO2'],'H2O':100*gas_streams.at[stream11,'H2O'],'Ar':100*gas_streams.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P02  = water_streams0.at[stream21,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P2   = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition0)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition0)
        T021   = prop.water_T_HP(self.H021,self.P02)
        T022   = prop.water_T_HP(self.H022,self.P02)
        T0np   = prop.water_TSS_P(self.P2)
        dTmin0 = min(T011-T0np,T012-T0np)
        dTmax0 = max(T011-T0np,T012-T0np)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition0)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition0)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition0)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition0)
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P2)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = prop.water_HSS_P(self.P2)
            T22 = prop.water_T_HP(H22,self.P2)
            G2 = Q/(H22-self.H21)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            dt = LMTD0/LMTD
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = prop.water_HSS_P(self.P2)
        T22 = prop.water_T_HP(H22,self.P2)
        G2 = Q/(H22-self.H21)
        return [T12,self.P1,H12,self.G1,T22,self.P2,H22,G2,Q]
###########################################################

import time
start_time = time.time()






iter1=50
Qq = n.zeros(iter1)
for j in range(iter1):
    print("--- %s сек. ---" % round((time.time() - start_time)))
    iter2=50
    Qqq = n.zeros(iter2)
    Ddd = n.zeros(iter2)
    for i in range(iter2):
        PEVD = heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS').calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        IVD  = vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD').calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        Qqq[i] = PEVD[8]+IVD[8]
        Ddd[i] = water_streams.at['PEVD-DROS','G']
        if i>0:
            if abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) < calctolerance and abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]) < calctolerance:
                break;
    EVD = heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD').calc()
    gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
    water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    Qq[j] = PEVD[8]+IVD[8]+EVD[8]
    if j>0:
        if abs((Qq[j-1]-Qq[j])/Qq[j-1]) < calctolerance:
            break;







iter1=50
Qq = n.zeros(iter1)
for j in range(iter1):
    iter2=50
    Qqq = n.zeros(iter2)
    Ddd = n.zeros(iter2)
    for i in range(iter2):
        PPND = heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB').calc()
        gas_streams.loc['PPND-IND','T':'G'] = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
    
        IND  = vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND').calc()
        gas_streams.loc['IND-GPK','T':'G'] = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
    
        water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
        Qqq[i] = PPND[8]+IND[8]
        Ddd[i] = water_streams.at['PPND-TURB','G']
        if i>0:
            print("--- %s сек. ---" % round((time.time() - start_time)))
            print(abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) , abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]))
            if abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) < calctolerance and abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]) < calctolerance:
                break;
    water_streams.loc['GPK-IND':'X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
    
    GPK = heatex('IND-GPK','GPK-out','X-GPK','GPK-IND').calc()
    gas_streams.loc['GPK-out','T':'G'] = [GPK[0],GPK[1],GPK[2],GPK[3]]
    water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]
    
    Qq[j] = GPK[8]+IND[8]+PPND[8]
    if j>0:
        if abs((Qq[j-1]-Qq[j])/Qq[j-1]) < calctolerance:
            break;



print("--- %s сек. ---" % round((time.time() - start_time)))


--- 0 сек. ---
--- 34 сек. ---
--- 68 сек. ---


In [ ]:
water_streams

In [ ]:
gas_streams

In [ ]:
print(prop.water_Q_HP(water_streams.at['GPK-IND','H'],water_streams.at['GPK-IND','P']))